In [1]:
import sys

if '..' not in sys.path:
    sys.path.insert(0, '..')

import glm
import plotter
import plotly.express as px
from pandas import DataFrame
from numpy import ndarray
import pandas as pd
import numpy as np
from plotly.graph_objects import Figure as PlotlyFigure
import statsmodels.api as sm
from scipy.stats import norm
# from statsmodels.discrete.discrete_model import Logit as LogisticRegressor
from statsmodels.formula.api import logit as LogisticRegressor
from statsmodels.discrete.discrete_model import Logit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from patsy import dmatrix, dmatrices
from collections import MutableMapping

/var/folders/s2/s80pr6vx43n4pkf0gbvg9zlh0000gn/T/ipykernel_20069/2029328539.py:22: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import MutableMapping


In [2]:
data = sm.datasets.get_rdataset("Duncan", "carData").data
data['prestige_ge_60'] = np.where(data['prestige'] >= 60, 1, 0)
data.head()

,type,income,education,prestige,prestige_ge_60
accountant,prof,62,86,82,1
pilot,prof,72,76,83,1
architect,prof,75,92,90,1
author,prof,55,90,76,1
chemist,prof,64,86,90,1


In [9]:
#construct design matrices according to formula
predictors = ["income", "education"]
target = "prestige_ge_60"
formula = f"{target} ~ 1 + {'+'.join(predictors)}"
y, X = dmatrices(formula, data=data)

#fit model
sklearn_model = LogisticRegression().fit(
    X=X,
    y=y.ravel(),
)


In [11]:
def confusion_matrix_scorer(y, y_pred):
    cm = confusion_matrix(y, y_pred)

    return {"tn": cm[0, 0], "fp": cm[0, 1], "fn": cm[1, 0], "tp": cm[1, 1]}


def get_score_results(y, y_pred, metrics):

    return {metric: scorer(y, y_pred) for metric, scorer in metrics.items()}


def flatten(d, parent_key="", sep="_"):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))

    return dict(items)


metrics = {
    "accuracy": accuracy_score,
    "precision": precision_score,
    "recall": recall_score,
    "f1": f1_score,
    "confusion_matrix": confusion_matrix_scorer,
}

pd.Series(flatten(get_score_results(y, sklearn_model.predict(X), metrics)))

accuracy                0.888889
precision               0.866667
recall                  0.812500
f1                      0.838710
confusion_matrix_tn    27.000000
confusion_matrix_fp     2.000000
confusion_matrix_fn     3.000000
confusion_matrix_tp    13.000000
dtype: float64